# Explore the Transformed Play by Play Data

## Setup - Paths - Dependencies

In [181]:
from config import recent_play_by_play, recent_clean_db

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

## Load Play by Play data
pbp_raw_df = pd.read_csv(recent_play_by_play, low_memory=False)


data_folder = os.path.join('..', 'data') # data folder
temp_folder = os.path.join('..', 'TEMP') # temp folder
pbp_folder = os.path.join(data_folder, 'vault', 'play_by_play_extracts') # pbp folder

# pbp_raw_df.head()


In [182]:
#### HOTFIX TO CLEAN 

### Drop games Exhibition games from dataset
# see two problem teams St Anselm and Assumption - if Game_ID contains either of the teams drop the row
pbp_raw_df = pbp_raw_df[~pbp_raw_df['Game_ID'].str.contains('St Anselm|Assumption')]

# Fix Problem with a few Primary_team values
# Substitute 'UCN' and 'UCN Gustafsson' with 'Connecticut'
pbp_raw_df.loc[pbp_raw_df['Primary_team'] == 'UCN', 'Primary_team'] = 'Connecticut'
pbp_raw_df.loc[pbp_raw_df['Primary_team'] == 'UCN Gustafsson', 'Primary_team'] = 'Connecticut'
# Replace the weird 'PRI de la' with 'Princeton'
pbp_raw_df.loc[pbp_raw_df['Primary_team'] == 'PRI de la', 'Primary_team'] = 'Princeton'


### HOTFIX TO ADD SECONDARY TEAM TO EVERY ROW
df = pbp_raw_df
# Create a new column 'Secondary_team' and initialize it with None
df['Secondary_team'] = None
# Loop through each row in the DataFrame
for index, row in df.iterrows():
    # Extract the game ID
    game_id = row['Game_ID']
    # Determine the home and away teams based on the game ID
    away_team = game_id.split('-')[3]
    home_team = game_id.split('-')[4]
    # Assign the opposite team to the 'Secondary_team' column based on the 'Primary_team'
    if row['Primary_team'] == home_team:
        df.at[index, 'Secondary_team'] = away_team
    else:
        df.at[index, 'Secondary_team'] = home_team

pbp_raw_df = df


### HOTFIX TO DEAL WITH St. Cloud State having the period when it shouldn't
# if St. Cloud State is in Primary_team or Secondary_team replace with 'St Cloud State'BaseExceptionGroup
pbp_raw_df.loc[pbp_raw_df['Primary_team'] == 'St. Cloud State', 'Primary_team'] = 'St Cloud State'
pbp_raw_df.loc[pbp_raw_df['Secondary_team'] == 'St. Cloud State', 'Secondary_team'] = 'St Cloud State'



In [183]:
# Rename to df for convenience
df = pbp_raw_df

# Filter for relevant events: Faceoffs and Goals
faceoff_events = df[df['Event_type'] == 'Faceoff']
goal_events = df[df['Event_type'] == 'Goal']

# Merge faceoff events with subsequent goal events within the same game and period
merged_df = pd.merge(
    faceoff_events[['Game_ID', 'Period', 'Time', 'Primary_team']],
    goal_events[['Game_ID', 'Period', 'Time', 'Primary_team']],
    on=['Game_ID', 'Period'],
    suffixes=('_faceoff', '_goal')
)

# Calculate the time difference between faceoff and goal
merged_df['time_diff'] = merged_df['Time_goal'] - merged_df['Time_faceoff']

# Filter only instances where the goal happens after the faceoff
merged_df = merged_df[merged_df['time_diff'] > 0]


# Count how often a goal is scored within 10 seconds and 5 seconds of a faceoff
goals_within_10s = (merged_df['time_diff'] <= 10).sum()
goals_within_5s = (merged_df['time_diff'] <= 5).sum()

# Count the number of times each team has scored a goal within 5 seconds of a faceoff
teams_scoring_within_5s = merged_df[merged_df['time_diff'] <= 5]['Primary_team_goal'].value_counts()

# Display results
# goals_within_10s, goals_within_5s, teams_scoring_within_5s


# Data Elporation

## Faceoff Danger
- Table of how many times a team has scored within 3,5, 7, and 10 seconds of a faceoff

In [184]:
# Recalculate the time difference properly
merged_df = pd.merge(
    faceoff_events[['Game_ID', 'Period', 'Time', 'Primary_team']],
    goal_events[['Game_ID', 'Period', 'Time', 'Primary_team']],
    on=['Game_ID', 'Period'],
    suffixes=('_faceoff', '_goal')
)

# Ensure that the goal occurs after the faceoff in time
merged_df = merged_df[merged_df['Time_goal'] > merged_df['Time_faceoff']]

# Calculate the correct time difference between faceoff and goal
merged_df['time_diff'] = merged_df['Time_goal'] - merged_df['Time_faceoff']

# Count goals per team in different time frames
teams_goal_counts = merged_df.groupby('Primary_team_goal')['time_diff'].agg(
    Scored_within_1s=lambda x: (x <= 1).sum(),
    Scored_within_2s=lambda x: (x <= 2).sum(),
    Scored_within_3s=lambda x: (x <= 3).sum(),
    Scored_within_4s=lambda x: (x <= 4).sum(),
    Scored_within_5s=lambda x: (x <= 5).sum(),
    Scored_within_6s=lambda x: (x <= 6).sum(),
    Scored_within_7s=lambda x: (x <= 7).sum(),
    Scored_within_8s=lambda x: (x <= 8).sum(),
    Scored_within_9s=lambda x: (x <= 9).sum(),
    Scored_within_10s=lambda x: (x <= 10).sum()
).reset_index()

# Add the total goals scored by each team
total_goals_per_team = goal_events['Primary_team'].value_counts().reset_index()
total_goals_per_team.columns = ['Primary_team_goal', 'Total_Goals']

# Merge with the team goal counts
teams_goal_counts = teams_goal_counts.merge(total_goals_per_team, on='Primary_team_goal', how='left')



In [185]:
teams_goal_counts.head()

,Primary_team_goal,Scored_within_1s,Scored_within_2s,Scored_within_3s,Scored_within_4s,Scored_within_5s,Scored_within_6s,Scored_within_7s,Scored_within_8s,Scored_within_9s,Scored_within_10s,Total_Goals
0,Air Force,0,0,0,2,2,2,4,5,6,8,55
1,Alaska,0,1,1,3,4,6,6,6,8,9,54
2,Alaska Anchorage,0,0,0,0,0,1,1,2,2,2,55
3,American Intl,0,0,0,0,1,1,1,2,2,4,57
4,Arizona State,0,0,0,0,1,1,1,1,1,2,90


In [186]:
# Calculate the percentage for each goal duration
for sec in range(1, 11):
    col = f'Scored_within_{sec}s'
    pct_col = f'{col}_pct'
    teams_goal_counts[pct_col] = teams_goal_counts[col] / teams_goal_counts['Total_Goals']

# Rearrange columns in an interleaved order: original value then its percentage
cols = ['Primary_team_goal', 'Total_Goals'] + [
    item for sec in range(1, 11)
    for item in (f'Scored_within_{sec}s', f'Scored_within_{sec}s_pct')
]
teams_goal_counts = teams_goal_counts[cols]

# Rename Primary_team_goal to Team
teams_goal_counts.rename(columns={'Primary_team_goal': 'Team'}, inplace=True)

In [187]:
# team_goal_counts.sample(10)

In [188]:
### Save to temp folder
# get todays date as string
today = pd.Timestamp.now().strftime('%Y-%m-%d')

from config import play_by_play_filename
# Remove .csv from filename
play_by_play_filename = play_by_play_filename.split('.')[0]
teams_goal_counts.to_csv(os.path.join(pbp_folder, f'faceoff_success_data_table_{play_by_play_filename}.csv'), index=False)




### Goals Allowed Just after faceoffs

In [189]:
## Reset df to a clean copy of the full pbp data
df = pbp_raw_df

# Fill the Secondary team column with the opposite team 
# If the Primary team is 'Home', the Secondary team is 'Away' and vice versa based on Game_ID
# Away is after third '-' in game_id. home is after 4th
# check primary team against game id and put opposite in secondary

# # Create a new column 'Secondary_team' and initialize it with None
# df['Secondary_team'] = None
# # Loop through each row in the DataFrame
# for index, row in df.iterrows():
#     # Extract the game ID
#     game_id = row['Game_ID']
#     # Determine the home and away teams based on the game ID
#     away_team = game_id.split('-')[3]
#     home_team = game_id.split('-')[4]
#     # Assign the opposite team to the 'Secondary_team' column based on the 'Primary_team'
#     if row['Primary_team'] == home_team:
#         df.at[index, 'Secondary_team'] = away_team
#     else:
#         df.at[index, 'Secondary_team'] = home_team


# Filter for relevant events: Faceoffs and Goals
faceoff_events = df[df['Event_type'] == 'Faceoff']
goal_events = df[df['Event_type'] == 'Goal']

# Check the first few rows to verify the new column
# df.head()

In [190]:

# Merge faceoff events with subsequent goal events within the same game and period
merged_allowed_df = pd.merge(
    faceoff_events[['Game_ID', 'Period', 'Time', 'Primary_team', 'Secondary_team']],
    goal_events[['Game_ID', 'Period', 'Time', 'Primary_team', 'Secondary_team']],
    on=['Game_ID', 'Period'],
    suffixes=('_faceoff', '_goal')
)

# Calculate the time difference between faceoff and goal
merged_allowed_df['time_diff'] = merged_allowed_df['Time_goal'] - merged_allowed_df['Time_faceoff']

# Filter only instances where the goal happens after the faceoff
merged_allowed_df = merged_allowed_df[merged_allowed_df['time_diff'] > 0]

merged_allowed_df.head()
# Count goals per team in different time frames
teams_goal_allowed = merged_allowed_df.groupby('Secondary_team_goal')['time_diff'].agg(
    within_1s=lambda x: (x <= 1).sum(),
    within_2s=lambda x: (x <= 2).sum(),
    within_3s=lambda x: (x <= 3).sum(),
    within_4s=lambda x: (x <= 4).sum(),
    within_5s=lambda x: (x <= 5).sum(),
    within_6s=lambda x: (x <= 6).sum(),
    within_7s=lambda x: (x <= 7).sum(),
    within_8s=lambda x: (x <= 8).sum(),
    within_9s=lambda x: (x <= 9).sum(),
    within_10s=lambda x: (x <= 10).sum()
).reset_index()

## LEGACY
# Add the total goals scored by each team 
# total_goals_per_team = goal_events['Secondary_team'].value_counts().reset_index()
# total_goals_per_team.columns = ['Secondary_team', 'Total_Goals']

# Merge with the team goal counts
 

# teams_goal_allowed.head()



In [191]:
# Add the season total of goals allowed by each team
total_goals_allowed_per_team = goal_events['Secondary_team'].value_counts().reset_index()
total_goals_allowed_per_team.columns = ['Secondary_team_goal', 'Total_Goals_Allowed']

# add the data into the teams_goal_allowed dataframe
teams_goal_allowed = teams_goal_allowed.merge(total_goals_allowed_per_team, on='Secondary_team_goal', how='left')


### Merge Two Datatables

In [192]:
# Rename columns for clarity in the allowed goals DataFrame
teams_goal_allowed.columns = ['Team', 'Allowed_within_1s', 'Allowed_within_2s', 'Allowed_within_3s',
                               'Allowed_within_4s', 'Allowed_within_5s', 'Allowed_within_6s', 'Allowed_within_7s',
                               'Allowed_within_8s', 'Allowed_within_9s', 'Allowed_within_10s', 'Total_Goals_Allowed']

# Check the data
teams_goal_allowed.sample(10)


,Team,Allowed_within_1s,Allowed_within_2s,Allowed_within_3s,Allowed_within_4s,Allowed_within_5s,Allowed_within_6s,Allowed_within_7s,Allowed_within_8s,Allowed_within_9s,Allowed_within_10s,Total_Goals_Allowed
33,Michigan,0,0,3,3,4,4,6,7,8,8,84
38,Minnesota State,0,0,0,0,0,0,0,1,1,1,40
17,Connecticut,0,0,0,2,4,7,7,7,9,11,60
40,Niagara,0,0,2,2,4,4,4,4,5,6,78
53,Robert Morris,0,0,2,2,2,2,4,4,5,5,79
61,Western Michigan,0,1,1,1,1,1,1,1,1,1,42
44,Notre Dame,0,0,1,1,1,3,6,7,7,9,78
10,Boston University,0,0,1,1,1,1,2,3,3,4,80
54,Sacred Heart,0,0,0,3,3,3,3,3,3,3,75
0,Air Force,0,2,2,2,2,2,3,3,5,9,76


In [193]:
# # Merge the allowed goals DataFrame with the total goals DataFrame
# face_off_goal_success = teams_goal_allowed.merge(teams_goal_counts, on='Team', how='left')
# Calculate the percentage for each goal duration
for sec in range(1, 11):
    col = f'Allowed_within_{sec}s'
    pct_col = f'{col}_pct'
    teams_goal_allowed[pct_col] = teams_goal_allowed[col] / teams_goal_allowed['Total_Goals_Allowed']


# Merge the allowed goals DataFrame with the total goals DataFrame
face_off_goal_success = teams_goal_allowed.merge(teams_goal_counts, on='Team', how='left')




### Save the Faceoff Success Table

In [194]:
## Save Final Table to Data Folder

face_off_goal_success.to_csv(os.path.join(pbp_folder, f'face_off_goal_success_{play_by_play_filename}.csv'), index=False)

# Examine table
# face_off_goal_success.head(10)

# Check info
# face_off_goal_success.info()

## AFTER PP SUCCESS - Within 10 seconds of end of PP

In [195]:
### Get all relevant rows from pbp data
# Filter for relevant events: PP-Start, PP-End and Goals
df = pbp_raw_df

# Filter for relevant events: PP-Start, PP-End and Goals
pp_start_events = df[df['Event_type'] == 'PP - Start']
pp_end_events = df[df['Event_type'] == 'PP - End']
goal_events = df[df['Event_type'] == 'Goal']
# Merge PP-Start events with subsequent goal events within the same game and period
merged_pp_df = pd.merge(
    pp_start_events[['Game_ID', 'Period', 'Time', 'Primary_team']],
    goal_events[['Game_ID', 'Period', 'Time', 'Primary_team']],
    on=['Game_ID', 'Period'],
    suffixes=('_pp_start', '_goal')
)

# Calculate the time difference between PP-Start and goal
merged_pp_df['time_diff'] = merged_pp_df['Time_goal'] - merged_pp_df['Time_pp_start']
# Filter only instances where the goal happens after the PP-End
merged_pp_df = merged_pp_df[merged_pp_df['time_diff'] > 0]
# Count goals per team in different time frames
teams_goal_pp = merged_pp_df.groupby('Primary_team_goal')['time_diff'].agg(
    Scored_within_1s=lambda x: (x <= 1).sum(),
    Scored_within_2s=lambda x: (x <= 2).sum(),
    Scored_within_3s=lambda x: (x <= 3).sum(),
    Scored_within_4s=lambda x: (x <= 4).sum(),
    Scored_within_5s=lambda x: (x <= 5).sum(),
    Scored_within_6s=lambda x: (x <= 6).sum(),
    Scored_within_7s=lambda x: (x <= 7).sum(),
    Scored_within_8s=lambda x: (x <= 8).sum(),
    Scored_within_9s=lambda x: (x <= 9).sum(),
    Scored_within_10s=lambda x: (x <= 10).sum(),
    Scored_within_15s=lambda x: (x <= 15).sum(),
    Scored_within_20s=lambda x: (x <= 20).sum(),
    Scored_within_30s=lambda x: (x <= 30).sum()
).reset_index()

# Add the total goals scored by each team
total_goals_per_team = goal_events['Primary_team'].value_counts().reset_index()
total_goals_per_team.columns = ['Primary_team_goal', 'Total_Goals']
# Merge with the team goal counts
teams_goal_pp = teams_goal_pp.merge(total_goals_per_team, on='Primary_team_goal', how='left')

# Calculate the percentage for each goal duration
# Define the seconds list to include 1-10, 15, 20, and 30 seconds
secs = list(range(1, 11)) + [15, 20, 30]

# Calculate the percentage for each goal duration
for sec in secs:
    col = f'Scored_within_{sec}s'
    pct_col = f'{col}_pct'
    teams_goal_pp[pct_col] = teams_goal_pp[col] / teams_goal_pp['Total_Goals']

# Rearrange columns in an interleaved order: original value then its percentage
cols = ['Primary_team_goal', 'Total_Goals'] + [
    item for sec in secs
    for item in (f'Scored_within_{sec}s', f'Scored_within_{sec}s_pct')
]
teams_goal_pp = teams_goal_pp[cols]

# Rename Primary_team_goal to Team
teams_goal_pp.rename(columns={'Primary_team_goal': 'Team'}, inplace=True)
# teams_goal_pp.head(10)
# teams_goal_pp.info(10)

In [196]:
### Count how many Power Play goals each team has scored by looking at the 'Goal_Conditions' column of pbp_raw_df 
# # if it contains POWER-PLAY it is a PP Goal


# Count how many Power Play goals each team has scored by looking at the 'Goal_Conditions' column of pbp_raw_df
# # if it contains POWER-PLAY it is a PP Goal
# Initialize a DataFrame to store the counts
pp_goal_counts = pd.DataFrame(columns=['Team', 'PP_Goals'])
# Loop through each team
for team in df['Primary_team'].unique():
    # Count the number of goals for the team that contain 'POWER-PLAY' in 'Goal_Conditions'
    pp_goals = df[(df['Primary_team'] == team) & (df['Goal_Conditions'].str.contains('POWER-PLAY', na=False))]
    pp_goal_counts = pd.concat([pp_goal_counts, pd.DataFrame({'Team': [team], 'PP_Goals': [len(pp_goals)]})])

# Reset the index
pp_goal_counts.reset_index(drop=True, inplace=True)
# Merge the PP goal counts with the main DataFrame
teams_goal_pp = teams_goal_pp.merge(pp_goal_counts, on='Team', how='left')
# Check the data
# teams_goal_pp.sample(10)




#### Goals Allowed Just after End of PP

In [197]:
### Do the same for allowed goals
# Merge PP-End events with subsequent goal events within the same game and period
merged_pp_allowed_df = pd.merge(
    pp_end_events[['Game_ID', 'Period', 'Time', 'Primary_team', 'Secondary_team']],
    goal_events[['Game_ID', 'Period', 'Time', 'Primary_team', 'Secondary_team']],
    on=['Game_ID', 'Period'],
    suffixes=('_pp_end', '_goal')
)

# Calculate the time difference between PP-End and goal
merged_pp_allowed_df['time_diff'] = merged_pp_allowed_df['Time_goal'] - merged_pp_allowed_df['Time_pp_end']
# Filter only instances where the goal happens after the PP-End
merged_pp_allowed_df = merged_pp_allowed_df[merged_pp_allowed_df['time_diff'] > 0]
# Count goals per team in different time frames
teams_goal_pp_allowed = merged_pp_allowed_df.groupby('Primary_team_goal')['time_diff'].agg(
    Allowed_within_1s=lambda x: (x <= 1).sum(),
    Allowed_within_2s=lambda x: (x <= 2).sum(),
    Allowed_within_3s=lambda x: (x <= 3).sum(),
    Allowed_within_4s=lambda x: (x <= 4).sum(),
    Allowed_within_5s=lambda x: (x <= 5).sum(),
    Allowed_within_6s=lambda x: (x <= 6).sum(),
    Allowed_within_7s=lambda x: (x <= 7).sum(),
    Allowed_within_8s=lambda x: (x <= 8).sum(),
    Allowed_within_9s=lambda x: (x <= 9).sum(),
    Allowed_within_10s=lambda x: (x <= 10).sum(),
    Allowed_within_15s=lambda x: (x <= 15).sum(),
    Allowed_within_20s=lambda x: (x <= 20).sum(),
    Allowed_within_30s=lambda x: (x <= 30).sum()
).reset_index()

# Rename Primary_team_goal to Team
teams_goal_pp_allowed.rename(columns={'Primary_team_goal': 'Team'}, inplace=True)
# Add the total goals allowed by each team
total_goals_allowed_per_team = goal_events['Secondary_team'].value_counts().reset_index()
total_goals_allowed_per_team.columns = ['Secondary_team', 'Total_Goals_Allowed']
# Rename to simply Team
total_goals_allowed_per_team.rename(columns={'Secondary_team': 'Team'}, inplace=True)

total_goals_allowed_per_team
# Merge with the team goal counts
teams_goal_pp_allowed = teams_goal_pp_allowed.merge(total_goals_allowed_per_team, on='Team', how='left')

# teams_goal_pp_allowed.head()
# teams_goal_pp_allowed.info()

# # look at mean min and max
# teams_goal_pp_allowed.describe()




In [198]:
## Calculate the percentage for each goal duration
# Define the seconds list to include 1-10, 15, 20, and 30 seconds
secs = list(range(1, 11)) + [15, 20, 30]

# Calculate the percentage for each goal duration
for sec in secs:
    col = f'Allowed_within_{sec}s'
    pct_col = f'{col}_pct'
    teams_goal_pp_allowed[pct_col] = teams_goal_pp_allowed[col] / teams_goal_pp_allowed['Total_Goals_Allowed']

# Rearrange columns in an interleaved order: original value then its percentage
cols = ['Team', 'Total_Goals_Allowed'] + [
    item for sec in secs
    for item in (f'Allowed_within_{sec}s', f'Allowed_within_{sec}s_pct')
]
teams_goal_pp_allowed = teams_goal_pp_allowed[cols]
# Merge the allowed goals DataFrame with the total goals DataFrame
face_off_allowed_goal_pp = teams_goal_pp_allowed.merge(teams_goal_pp, on='Team', how='left')
# face_off_allowed_goal_pp.head(10)

In [199]:
## Count the number of PP goals allowed by each team and add to the DataFrame
# Initialize a DataFrame to store the counts
pp_goal_allowed_counts = pd.DataFrame(columns=['Team', 'PP_Goals_Allowed'])
# Loop through each team
for team in df['Secondary_team'].unique():
    # Count the number of goals for the team that contain 'POWER-PLAY' in 'Goal_Conditions'
    pp_goals_allowed = df[(df['Secondary_team'] == team) & (df['Goal_Conditions'].str.contains('POWER-PLAY', na=False))]
    pp_goal_allowed_counts = pd.concat([pp_goal_allowed_counts, pd.DataFrame({'Team': [team], 'PP_Goals_Allowed': [len(pp_goals_allowed)]})])
# Reset the index
pp_goal_allowed_counts.reset_index(drop=True, inplace=True)

# Merge the PP goal counts with the main DataFrame
teams_goal_pp_allowed = teams_goal_pp_allowed.merge(pp_goal_allowed_counts, on='Team', how='left')
# Check the data
# teams_goal_pp_allowed.sample(10)

In [200]:
## Merge the Scored and allowed datasets into a single final table
# Merge the scored and allowed DataFrames
post_power_play_success = teams_goal_pp_allowed.merge(teams_goal_pp, on='Team', how='left')
# Check the data
# post_power_play_success.sample(10)

#### Save Post PP Data to CSV File in Vault

In [201]:
### Save Final Table to PBP Extract Folder
post_power_play_success.to_csv(os.path.join(pbp_folder, f'post_power_play_success_{play_by_play_filename}.csv'), index=False)

In [202]:
post_power_play_success.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 57 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Team                    64 non-null     object 
 1   Total_Goals_Allowed     63 non-null     float64
 2   Allowed_within_1s       64 non-null     int64  
 3   Allowed_within_1s_pct   63 non-null     float64
 4   Allowed_within_2s       64 non-null     int64  
 5   Allowed_within_2s_pct   63 non-null     float64
 6   Allowed_within_3s       64 non-null     int64  
 7   Allowed_within_3s_pct   63 non-null     float64
 8   Allowed_within_4s       64 non-null     int64  
 9   Allowed_within_4s_pct   63 non-null     float64
 10  Allowed_within_5s       64 non-null     int64  
 11  Allowed_within_5s_pct   63 non-null     float64
 12  Allowed_within_6s       64 non-null     int64  
 13  Allowed_within_6s_pct   63 non-null     float64
 14  Allowed_within_7s       64 non-null     int6

In [203]:
total_goals_allowed_per_team.head(50)
total_goals_allowed_per_team.tail(20)

,Team,Total_Goals_Allowed
44,Bentley,59
45,Ohio State,58
46,Quinnipiac,57
47,Dartmouth,57
48,Providence,57
49,Michigan State,57
50,Clarkson,56
51,Arizona State,56
52,Michigan Tech,54
53,Omaha,54
